In [56]:
import pandas as pd

In [57]:
data = pd.read_excel("survey.xlsx");
# We drop the cells including the text of the questions
data = data.drop(0, axis =0)

In [58]:
# We create a dataframe containing only the questions that both treatment and control group have in common, and which we'll use as covariates for the study
data_shared = data[['Q3', 'Q4', 'Q5', 'Q9', 'Q8', 'Q10', 'Q11', 'Q15', 'Q16','Q21']]

In [59]:
# We ensure that the columns of the dataframe are in "int" format
data_shared = data_shared.astype(int)

# We rename the columns of the dataframe to assign descriptive names
data_shared.columns = ["age", "gender", "scholarship","1st_year", "gpa", "1st_time","taste", "importance", "expected_grade","knowledge"]

In [60]:
data_shared.head()

,age,gender,scholarship,1st_year,gpa,1st_time,taste,importance,expected_grade,knowledge
1,2,1,1,1,6,1,4,3,6,1
2,2,2,2,1,8,1,3,4,5,1
3,2,1,2,2,6,1,3,3,3,1
4,2,2,2,1,8,1,3,3,3,2
5,6,2,2,1,3,1,3,3,6,1


In [61]:
# We reformat the values of our questions to make them correspond to what we actually care about
data_shared["age"] = data_shared["age"]+17 # As the minimum age was 18, labelled as entry 1, we add 17
data_shared["gender"] = data_shared["gender"]-1 # We reformat the variable so that it ranges between 0 and 1
data_shared["scholarship"] = 1-(data_shared["scholarship"]-1) # We reformat the variable so that it ranges between 0 and 1
data_shared["1st_time"] = 1-(data_shared["1st_time"]-1) # We reformat the variable so that it ranges between 0 and 1
data_shared["gpa"] = data_shared["gpa"]+17 # As the minimum gpa was 18, labelled as entry 1, we add 17
data_shared["1st_year"] = 1-(data_shared["1st_year"]-1) # We reformat the variable so that it ranges between 0 and 1
data_shared["expected_grade"] = data_shared["expected_grade"]+16 # As the minimum is less than 18, so we use 17 as a benchmark for that
data_shared["knowledge"] = data_shared["knowledge"]-1 # We reformat the variable so that it ranges between 0 and 1

In [62]:
grade_goal_y = data[["Treatment 1", "Control 1"]]
grade_goal_y = grade_goal_y+17 # as the minimum grade is 18, which is labelled as 1

In [63]:
# We add a column indicating if an individual was in the treatment or in the control group
data_shared["treated"] = grade_goal_y['Treatment 1'].apply(lambda x: 0 if pd.isna(x) else 1)

In [64]:
y = pd.DataFrame()

In [65]:
# Now that we know who's in the treatment and who's in the control group, we merge the treatment and control group ina single column
y["grade_goal"] = grade_goal_y['Treatment 1'].apply(lambda x: 0 if pd.isna(x) else x)+grade_goal_y["Control 1"].fillna(0)
y["confidence"] = data['Treatment 2'].apply(lambda x: 0 if pd.isna(x) else x)+data["Control 2"].fillna(0)
y["anxiety"] = 6-(data['Treatment 3'].apply(lambda x: 0 if pd.isna(x) else x)+data["Control 3"].fillna(0)) # we renormalize the variables so that 1 is the lowest degree of anxiety, while 5 is the highest
y["time"] = data['Treatment 4'].apply(lambda x: 0 if pd.isna(x) else x)+data["Control 4"].fillna(0) #1: a few days, 6: more than a month; the rest in between ranges from 1 to 4 weeks
y["exam_date"] = data['Treatment 5'].apply(lambda x: 0 if pd.isna(x) else x)+data["Control 5"].fillna(0) # From first exam data to fourth exam date

In [66]:
y.head()

,grade_goal,confidence,anxiety,time,exam_date
1,26,4,2,5,1
2,25,3,5,4,1
3,20,4,4,5,1
4,20,3,4,6,1
5,25,3,3,5,2


In [67]:
data_shared.head()

,age,gender,scholarship,1st_year,gpa,1st_time,taste,importance,expected_grade,knowledge,treated
1,19,0,1,1,23,1,4,3,22,0,0
2,19,1,0,1,25,1,3,4,21,0,1
3,19,0,0,0,23,1,3,3,19,0,0
4,19,1,0,1,25,1,3,3,19,1,1
5,23,1,0,1,20,1,3,3,22,0,0


In [68]:
y.to_csv("dep_y.csv")
data_shared.to_csv("indep_x.csv")